In [1]:
import openai
import os
from dotenv import load_dotenv
import json
import re

load_dotenv()  # take environment variables from .env.
openai.api_key = os.getenv('OPENAI_API_KEY')



In [2]:
cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}

In [3]:
# one model, one question, certain temperature. Return an inclination on certain dimension of mbti.
def get_model_answer(model, question, temperature=0.5, add_sys_prompt=''):
    # You can force model to change mbti via 'add_sys_prompt', such as, "Your personality is ????""
    system_prompt = 'You can only anwser one letter, A or B.' + add_sys_prompt 
    
    response = openai.ChatCompletion.create(
        model=model,
        temperature=temperature,
        response_format={ "type": "text" },
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
    )
    
    cost_info['prompt_tokens'] += response.usage.prompt_tokens
    cost_info['input_tokens'] += response.usage.completion_tokens
    
    if model == 'gpt-3.5-turbo':
        cost_info['cost'] += response.usage.total_tokens * 0.5/1000000
    elif model == 'gpt-4o':
        cost_info['cost'] += response.usage.total_tokens * 5/1000000
    
    def extract_A_or_B(input_string):
        # 使用正则表达式匹配大写字母A或B
        match = re.search(r'[AB]', input_string)
        if match:
            return match.group()
        else:
            return None
    
    choice = response.choices[0].message.content
    choice = extract_A_or_B(choice)
    return choice

In [4]:
# mbti for one model
def get_mbti(model, temperature=0.5, add_sys_prompt=''):
    mbti_questions = json.load(
        open('mbti_questions.json', 'r', encoding='utf8')
    )

    cur_model_score = {
            'E': 0,
            'I': 0,
            'S': 0,
            'N': 0,
            'T': 0,
            'F': 0,
            'J': 0,
            'P': 0
        }

    for i in range(3): 
        for q in mbti_questions.values():
            question = q['question']
            res = get_model_answer(
                model=model, 
                temperature=temperature, 
                question=question,
                add_sys_prompt=add_sys_prompt
            )
            # print(res)
            mbti_choice = q[res]
            cur_model_score[mbti_choice] += 1
        
    e_or_i = 'E' if cur_model_score['E'] > cur_model_score['I'] else 'I'
    s_or_n = 'S' if cur_model_score['S'] > cur_model_score['N'] else 'N'
    t_or_f = 'T' if cur_model_score['T'] > cur_model_score['F'] else 'F'
    j_or_p = 'J' if cur_model_score['J'] > cur_model_score['P'] else 'P'

    result = {
        'model': model,
        'details': cur_model_score,
        'res': ''.join([e_or_i, s_or_n, t_or_f, j_or_p])
    }
    
    return(result)
    

---

## Test


# Change gpt3.5 and 4o LLMs' mbti via explicit prompt
# ISFP__ENTJ 3.5
# ESFP_INTJ  4.0


# 3.5

In [43]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
personalities = json.load(
    open('prompt_explicit.json', 'r', encoding='utf8')
)

cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}
exp_prompt_3 = personalities["ISFP_3"].get('traits','')
print(exp_prompt_3)

result3 = get_mbti(
    model='gpt-3.5-turbo', 
    add_sys_prompt=exp_prompt_3
)
result3 = result3 | cost_info



KeyError: 'ISFP_3'

In [35]:
exp_prompt_3_mbti = {}
exp_prompt_3_mbti["imp_prompt_ISFP"] = result3

In [36]:
exp_prompt_3_mbti

{'Exp_prompt_ISFP': {'model': 'gpt-3.5-turbo',
  'details': {'E': 7,
   'I': 56,
   'S': 22,
   'N': 59,
   'T': 21,
   'F': 48,
   'J': 33,
   'P': 33},
  'res': 'INFP',
  'prompt_tokens': 17979,
  'input_tokens': 606,
  'cost': 0.009292500000000002}}

# 4o

In [45]:
personalities = json.load(
    open('prompt_explicit.json', 'r', encoding='utf8')
)

exp_prompt_4_mbti = {}

cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}
exp_prompt = personalities["ESFP_4o"].get('traits','')
print(exp_prompt)

result4 = get_mbti(
    model='gpt-4o', 
    add_sys_prompt=exp_prompt
)
result4 = result4 | cost_info

You have a personality type with the Extraverted, Observant, Feeling, and Prospecting traits.


In [42]:
personalities

{'ISFP': {'traits': 'You have a personality type with the Introverted, Observant, Feeling, and Prospecting traits.'},
 'ESFP': {'traits': 'You have a personality type with the Extraverted, Observant, Feeling, and Prospecting traits.'}}

In [47]:
exp_prompt_4_mbti={}
exp_prompt_4_mbti["ESFP"] = result4
exp_prompt_4_mbti

{'ESFP': {'model': 'gpt-4o',
  'details': {'E': 63,
   'I': 0,
   'S': 42,
   'N': 36,
   'T': 6,
   'F': 66,
   'J': 5,
   'P': 61},
  'res': 'ESFP',
  'prompt_tokens': 18210,
  'input_tokens': 279,
  'cost': 0.0924450000000001}}

In [48]:
# save to file
CONDI_SAVE_PATH = 'explicit_res.json'
condition_mbti = {}
condition_mbti['GPT-3.5'] = exp_prompt_3_mbti
condition_mbti['GPT-4o'] = exp_prompt_4_mbti

with open(CONDI_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(condition_mbti, json_file, indent=4)

 # Implicit prompt

In [52]:
# 3.5
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
personalities = json.load(
    open('prompt_implicit.json', 'r', encoding='utf8')
)

cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}
imp_prompt_3 = personalities["ISFP_3"].get('Q&A','')
print(imp_prompt_3)


When you are going out for a whole day, you would:\nA. Plan things to do and the time\nB. Not make plans, just go with the flow\nAnswer:B\n\nYou are usually:\nA. A sociable person\nB. A quiet person\nAnswer:B\n\nYou would make friends with people who:\nA. Often come up with new ideas\nB. Are down-to-earth\nAnswer:B\n\nYou often tend to:\nA. Let your emotions control your reason\nB. Let your reason control your emotions\nAnswer:A


In [ ]:

im_result3 = get_mbti(
    model='gpt-3.5-turbo', 
    add_sys_prompt=imp_prompt_3
)
im_result3 = im_result3 | cost_info


In [58]:

imp_prompt_3_mbti = {}
imp_prompt_3_mbti["imp_prompt_ISFP"] = im_result3
imp_prompt_3_mbti

{'imp_prompt_ISFP': {'model': 'gpt-3.5-turbo',
  'details': {'E': 43,
   'I': 20,
   'S': 35,
   'N': 46,
   'T': 14,
   'F': 55,
   'J': 39,
   'P': 27},
  'res': 'ENFJ',
  'prompt_tokens': 77030,
  'input_tokens': 491,
  'cost': 0.03876050000000004}}

In [55]:
# 4o
personalities = json.load(
    open('prompt_implicit.json', 'r', encoding='utf8')
)

imp_prompt_4_mbti = {}

cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}
imp_prompt = personalities["ESFP_4o"].get('Q&A','')
print(imp_prompt)

im_result4 = get_mbti(
    model='gpt-4o', 
    add_sys_prompt=imp_prompt
)



When you are going out for a whole day, you would:\nA. Plan things to do and the time\nB. Not make plans, just go with the flow\nAnswer:B\n\nYou are usually:\nA. A sociable person\nB. A quiet person\nAnswer:A\n\nYou would make friends with people who:\nA. Often come up with new ideas\nB. Are down-to-earth\nAnswer:B\n\nYou often tend to:\nA. Let your emotions control your reason\nB. Let your reason control your emotions\nAnswer:A


In [57]:
im_result4 = im_result4 | cost_info
personalities
imp_prompt_4_mbti={}
imp_prompt_4_mbti["ESFP"] = im_result4
imp_prompt_4_mbti


{'ESFP': {'model': 'gpt-4o',
  'details': {'E': 44,
   'I': 19,
   'S': 39,
   'N': 42,
   'T': 15,
   'F': 54,
   'J': 16,
   'P': 50},
  'res': 'ENFP',
  'prompt_tokens': 44715,
  'input_tokens': 279,
  'cost': 0.2249700000000002}}

In [59]:


# save to file
CONDI_SAVE_PATH = 'implicit_res.json'
condition_mbti = {}
condition_mbti['GPT-3.5'] = imp_prompt_3_mbti
condition_mbti['GPT-4o'] = imp_prompt_4_mbti

with open(CONDI_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(condition_mbti, json_file, indent=4)